Nama : Akmal Shahib Maulana

# **Kegiatan Modul 6**

Content Based Filtering : Anime

Tulislan code sesuai instruksi pada sel yang telah disediakan

## Downlaod dan extract dataset dari situs [kaggle.com](https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020)

In [ ]:
!kaggle datasets download -d hernan4444/anime-recommendation-database-2020

Dataset URL: https://www.kaggle.com/datasets/hernan4444/anime-recommendation-database-2020
License(s): CC0-1.0
100% 659M/661M [00:23<00:00, 30.2MB/s]
100% 661M/661M [00:23<00:00, 29.3MB/s]


In [ ]:
!unzip /content/anime-recommendation-database-2020.zip

Archive:  /content/anime-recommendation-database-2020.zip
  inflating: anime.csv               
  inflating: anime_with_synopsis.csv  
  inflating: animelist.csv           
  inflating: html folder/html/1/details.html  
  inflating: html folder/html/1/pictures.html  
  inflating: html folder/html/1/recomendations.html  
  inflating: html folder/html/1/reviews_1.html  
  inflating: html folder/html/1/reviews_10.html  
  inflating: html folder/html/1/reviews_11.html  
  inflating: html folder/html/1/reviews_12.html  
  inflating: html folder/html/1/reviews_13.html  
  inflating: html folder/html/1/reviews_14.html  
  inflating: html folder/html/1/reviews_15.html  
  inflating: html folder/html/1/reviews_16.html  
  inflating: html folder/html/1/reviews_17.html  
  inflating: html folder/html/1/reviews_18.html  
  inflating: html folder/html/1/reviews_19.html  
  inflating: html folder/html/1/reviews_2.html  
  inflating: html folder/html/1/reviews_20.html  
  inflating: html folder/html/

## Import library dan load dataset

In [ ]:
import pandas as pd

In [ ]:
# Untuk tugas ini cukup gunakan data "anime_with_synopsis.csv"
# Tampilkan 5 data teratas
df = pd.read_csv('/content/anime_with_synopsis.csv')
df.head()

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...


## Tangani data yang kosong (missing value)

In [ ]:
########################
# Preprocessing
########################
print(df.isna().sum())

MAL_ID       0
Name         0
Score        0
Genres       0
sypnopsis    8
dtype: int64


In [ ]:
df

,MAL_ID,Name,Score,Genres,sypnopsis
0,1,Cowboy Bebop,8.78,"Action, Adventure, Comedy, Drama, Sci-Fi, Space","In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: Tengoku no Tobira,8.39,"Action, Drama, Mystery, Sci-Fi, Space","other day, another bounty—such is the life of ..."
2,6,Trigun,8.24,"Action, Sci-Fi, Adventure, Comedy, Drama, Shounen","Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,7.27,"Action, Mystery, Police, Supernatural, Drama, ...",ches are individuals with special powers like ...
4,8,Bouken Ou Beet,6.98,"Adventure, Fantasy, Shounen, Supernatural",It is the dark century and the people are suff...
...,...,...,...,...,...
16209,48481,Daomu Biji Zhi Qinling Shen Shu,Unknown,"Adventure, Mystery, Supernatural",No synopsis information has been added to this...
16210,48483,Mieruko-chan,Unknown,"Comedy, Horror, Supernatural",ko is a typical high school student whose life...
16211,48488,Higurashi no Naku Koro ni Sotsu,Unknown,"Mystery, Dementia, Horror, Psychological, Supe...",Sequel to Higurashi no Naku Koro ni Gou .
16212,48491,Yama no Susume: Next Summit,Unknown,"Adventure, Slice of Life, Comedy",New Yama no Susume anime.


Hasil penanganan missing value

In [ ]:
def preprocessor(df):
    df.dropna(inplace = True)
    df.drop("Score", axis = 1, inplace = True)
preprocessor(df)

In [ ]:
print(df.isna().sum())

MAL_ID       0
Name         0
Genres       0
sypnopsis    0
dtype: int64


## Selanjutnya lakukan feature extraction pada fitur "Genres" menggunakan TF-IDF

In [ ]:
########################
# Creating TF-IDF Matrix
########################

from sklearn.feature_extraction.text import TfidfVectorizer

data = df['Genres']
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(data)
tfidf_matrix.shape

(16206, 46)

## Menghitung cosine similarity dari TF-IDF diatas

In [ ]:
########################
# Creating Cosine Similarity Matrix
########################

from sklearn.metrics.pairwise import cosine_similarity
cos_sim = cosine_similarity(tfidf_matrix)
cos_sim.shape

(16206, 16206)

## Membuat Content Based Recomender

In [ ]:
########################
# Creating Content Based Recommender
########################

def content_based_recommender(df, movie_name="Cowboy Bebop", rec_count=5):
    indices = pd.Series(df.index, index=df["Name"])
    indices = indices[~indices.index.duplicated(keep="last")]
    movie_index = indices[movie_name]
    similarity_score = pd.DataFrame(cos_sim[movie_index], columns=["score"])
    similar_animes = similarity_score.sort_values(by="score", ascending=False).iloc[0:rec_count].index
    recommendations = df.loc[similar_animes, ["Name", "Genres"]]
    return recommendations

## Mencoba Content Based Recomender

Mempilkan 5 Judul Anime dengan yang memiliki nilai kemiripan tertinggi berdasarkan genre dari kata kunci judul anime yang dimasukkan

In [ ]:
# hasil rekomendasi anime dengan keyword "Paprika"
content_based_recommender(df, movie_name = "Paprika")

,Name,Genres
1691,Paprika,"Dementia, Fantasy, Horror, Mystery, Psychologi..."
15167,Xue Ying Ling Zhu Zhi Qi Yu Pian,"Action, Adventure, Historical, Romance, Martia..."
16203,Shikaru Neko,Slice of Life
824,Higurashi no Naku Koro ni,"Mystery, Dementia, Horror, Psychological, Supe..."
13263,"Chiisana Tayori, Ookina Tayori.","Drama, Music"


In [ ]:
# hasil rekomendasi anime dengan keyword "Monster"
content_based_recommender(df, movie_name = "Monster")

,Name,Genres
9,Monster,"Drama, Horror, Mystery, Police, Psychological,..."
6598,Higurashi no Naku Koro ni Kaku: Outbreak,"Mystery, Horror, Psychological, Thriller"
2495,Death Note: Rewrite,"Mystery, Police, Psychological, Supernatural, ..."
13402,Jingai-san no Yome,"Comedy, Romance, Fantasy, Josei"
10449,B: The Beginning,"Action, Mystery, Police, Psychological, Supern..."


In [ ]:
# hasil rekomendasi anime dengan keyword "Naruto"
content_based_recommender(df, movie_name = "Naruto")

,Name,Genres
1508,Naruto: Shippuuden,"Action, Adventure, Comedy, Super Power, Martia..."
10,Naruto,"Action, Adventure, Comedy, Super Power, Martia..."
11638,3-Nen D-Gumi Glass no Kamen: Tobidase! Watashi...,"Comedy, Parody, School"
6158,Naruto: Shippuuden Movie 6 - Road to Ninja,"Action, Adventure, Super Power, Martial Arts, ..."
5518,Naruto: Honoo no Chuunin Shiken! Naruto vs. Ko...,"Action, Adventure, Martial Arts, Shounen, Supe..."
